# MixedStream objects and thermodynamic equilibrium

MixedStream is an extention of [Stream](https://biosteam.readthedocs.io/en/latest/Stream.html) with 's' (solid), 'l' (liquid), 'L' (LIQUID), and 'g' (gas) flow rates. The upper case 'LIQUID' denotes that it is a distinct phase from 'liquid'.

### Create MixedStream Objects

Before initializing MixedStream objects, first set the species:

In [1]:
import biosteam as bst
bst.MixedStream.species = bst.Species('Water', 'Ethanol')

Initialize with an ID and optionally T and P. Then you can set flow rates for different phases:

In [2]:
ms1 = bst.MixedStream(ID='ms1', T=351, P=101325)
ms1.setflow('l', Ethanol=1, units='kmol/hr')
ms1.setflow('g', Ethanol=2) # Assuming kmol/hr
ms1.getflow('l', 'Ethanol', 'Water')

array([1., 0.])

You can **view** flow rates differently by setting the units for show.

In [3]:
ms1.show(flow='kg/hr', T='degC')

MixedStream: ms1
 phase: 'lg', T: 77.85 degC, P: 101325 Pa
         species  kg/hr
 liquid: Ethanol  46.1

 vapor:  Ethanol  92.1


### Get and set flow rates

Flow rates are stored in solid_mol, liquid_mol, LIQUID_mol, and vapor_mol arrays:

In [4]:
ms1.solid_mol

array([0., 0.])

In [5]:
ms1.liquid_mol

array([0., 1.])

In [6]:
ms1.LIQUID_mol

array([0., 0.])

In [7]:
ms1.vapor_mol

array([0., 2.])

Assign flows using these properties:

In [8]:
ms1.liquid_mol[:] = [2, 1]
ms1.vapor_mol[1] = 3
ms1

MixedStream: ms1
 phase: 'lg', T: 351 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    2
         Ethanol  1

 vapor:  Ethanol  3


Mass and volumetric flow rates are also availabe as [property_array](https://array-collections.readthedocs.io/en/latest/property_array.html) objects

In [9]:
ms1.liquid_mass

property_array([36.031, 46.068])

In [10]:
ms1.liquid_vol

property_array([0.037, 0.063])

Assign flows through the mass or vol property arrays:

In [11]:
# Set gas phase specie flow rate by index
ms1.vapor_mass[0] = 10
ms1.show()

MixedStream: ms1
 phase: 'lg', T: 351 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    2
         Ethanol  1

 vapor:  Water    0.555
         Ethanol  3


In [12]:
# Set liquid phase flow rates assuming same order as in species object
ms1.liquid_vol[:] = [0.1, 0.2]
ms1.show()

MixedStream: ms1
 phase: 'lg', T: 351 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    5.39
         Ethanol  3.19

 vapor:  Water    0.555
         Ethanol  3


### Single phase flow properties

Add 'net' to get the net flow rate 

In [13]:
ms1.liquid_molnet

8.581060526320748

In [14]:
ms1.vapor_massnet

array(148.20532, dtype=object)

In [15]:
ms1.solid_volnet

array(0.0, dtype=object)

Add 'frac' to get the composition

In [16]:
ms1.vapor_molfrac

tuple_array([0.156, 0.844])

In [17]:
ms1.liquid_massfrac

tuple_array([0.3976676602743484, 0.6023323397256516], dtype=object)

In [18]:
ms1.solid_volfrac

tuple_array([0.5, 0.5])

Note: When a phase has no flow rate, all specie fractions will be equal.

### Overall flow properties

In [19]:
ms1.mol

tuple_array([5.944, 6.192])

In [20]:
ms1.mass

tuple_array([107.085, 285.257])

In [21]:
ms1.vol

tuple_array([15.821831898239994, 84.2115640153857], dtype=object)

Note that overall flow rate properties 'molnet', 'massnet', 'volnet', 'molfrac', 'massfrac', and 'volfrac' are also available.

### Material and thermodynamic properties

Access the same properties as Stream objects:

In [22]:
ms1.H # Enthalpy (kJ/hr) 

204334.2743237495

In [23]:
ms1.rho # Density (kg/m3)

3.9221069813948026

A dictionary of units of measure is available:

In [24]:
ms1.units # See documentation for more information

{'MW': 'g/mol',
 'mass': 'kg/hr',
 'mol': 'kmol/hr',
 'vol': 'm^3/hr',
 'massnet': 'kg/hr',
 'molnet': 'kmol/hr',
 'volnet': 'm^3/hr',
 'massfrac': 'kg/kg',
 'molfrac': 'kmol/kmol',
 'volfrac': 'm^3/m^3',
 'T': 'K',
 'P': 'Pa',
 'H': 'kJ/hr',
 'S': 'kJ/hr',
 'G': 'kJ/hr',
 'U': 'kJ/hr',
 'A': 'kJ/hr',
 'Hf': 'kJ/hr',
 'C': 'kJ/K/hr',
 'Vm': 'm^3/mol',
 'Cpm': 'J/mol/K',
 'Cp': 'J/g/K',
 'rho': 'kg/m^3',
 'rhom': 'mol/m^3',
 'nu': 'm^2/s',
 'mu': 'Pa*s',
 'sigma': 'N/m',
 'k': 'W/m/K',
 'alpha': 'm^2/s'}

### Vapor-liquid equilibrium

Set temperature and pressure:

In [25]:
ms2 = bst.MixedStream('ms2', T=353.88)
bst.MixedStream.lazy_energy_balance = False
ms2.setflow('g', Water=1, Ethanol=2)
ms2.setflow('l', Water=2, Ethanol=1)
ms2.VLE(T=353.88, P=101325)
ms2

MixedStream: ms2
 phase: 'lg', T: 353.88 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    1.84
         Ethanol  1.16

 vapor:  Water    1.16
         Ethanol  1.84


Set pressure and duty:

In [26]:
ms2.VLE(P=101325, Q=0)
ms2

MixedStream: ms2
 phase: 'lg', T: 353.88 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    1.84
         Ethanol  1.16

 vapor:  Water    1.16
         Ethanol  1.84


Set vapor fraction and pressure:

In [27]:
ms2.VLE(V=0.5, P=101325)
ms2

MixedStream: ms2
 phase: 'lg', T: 353.88 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    1.84
         Ethanol  1.16

 vapor:  Water    1.16
         Ethanol  1.84


Set vapor fraction and temperature:

In [28]:
ms2.VLE(V=0.5, T=353.88)
ms2

MixedStream: ms2
 phase: 'lg', T: 353.88 K, P: 101331 Pa
         species  kmol/hr
 liquid: Water    1.84
         Ethanol  1.16

 vapor:  Water    1.16
         Ethanol  1.84


Set temperature and duty:

In [29]:
ms2.VLE(Q=0, T=353.88)
ms2

MixedStream: ms2
 phase: 'lg', T: 353.88 K, P: 101331 Pa
         species  kmol/hr
 liquid: Water    1.84
         Ethanol  1.16

 vapor:  Water    1.16
         Ethanol  1.84


It is also possible to set light and heavy keys that are not used to calculate equilibrium using the `LNK` and `HNK` key word arguments.

### Liquid-liquid equilibrium

Initialize with MixedStream object with water and octane:

In [30]:
# Make stream with hydrophobic species
ms3 = bst.MixedStream('ms3', species=bst.Species('Water', 'Octane'))
ms3.setflow('l', (2, 2))
ms3

MixedStream: ms3
 phase: 'l', T: 298.15 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    2
         Octane   2


Adiabatic and isobaric conditions:

In [31]:
# Must set liquid-LIQUID guess splits
ms3.LLE() 
ms3

MixedStream: ms3
 phase: 'lL', T: 298.15 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    1.98
         Octane   2.49e-05

 LIQUID: Water    0.0167
         Octane   2


Note that `LLE` assumes no significat temperature change with phase partitioning, resulting in constant temperature.

Isothermal and isobaric conditions:

In [32]:
# Must set liquid-LIQUID guess splits
ms3.LLE(T=340) 
ms3

MixedStream: ms3
 phase: 'lL', T: 340 K, P: 101325 Pa
         species  kmol/hr
 liquid: Water    1.94
         Octane   5.37e-05

 LIQUID: Water    0.0627
         Octane   2
